In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import copy

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## 加载数据

In [6]:
# 数据处理、增强
data_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

ct_dataset = datasets.ImageFolder(root=r'data', transform=data_transform)

data_loader = torch.utils.data.DataLoader(ct_dataset, batch_size=8, shuffle=True, num_workers=0)

data_size = len(ct_dataset)

images, classes = ct_dataset[1]
print(len(ct_dataset))
print(ct_dataset.class_to_idx)
print(ct_dataset)

3692
{'negative': 0, 'positive': 1}
Dataset ImageFolder
    Number of datapoints: 3692
    Root Location: data
    Transforms (if any): Compose(
                             RandomHorizontalFlip(p=0.5)
                             ToTensor()
                         )
    Target Transforms (if any): None


## 定义网络
删除resnet18的平均池化层和全连接层，换成自己的

In [3]:
class Net(nn.Module):
    def __init__(self , model):
        super(Net, self).__init__()
        
        # 不更新预训练模型的梯度
        for param in model.parameters():
            param.requires_grad = False
            
        #摘掉model的后两层
        self.resnet_layer = nn.Sequential(*list(model.children())[:-2])
        # 添加的网络层
        self.finetune_layer = nn.Sequential(
            nn.MaxPool2d(2, 2),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True ),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(output_size=(1, 1)),
        )
        
        self.fc = nn.Linear(512, 2)
        
        
    def forward(self, x):
        x = self.resnet_layer(x)
        x = self.finetune_layer(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 定义训练函数

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    
    for epoch in range(num_epochs):
        print('-*-' * 20)
        print('epoch {}/{}'.format(epoch, num_epochs - 1))
        
        scheduler.step() # Decays the learning rate 
        model.train() # Set model to training mode
        
        running_loss = 0.0
        running_corrects = 0
        
        # Iterate over data.
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1) # torch.max返回的格式时(value, index)
                # print(labels, preds)
                loss = criterion(outputs, preds)
            
                loss.backward()
                optimizer.step()
        
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds==labels.data)
        
        epoch_loss = running_loss / data_size
        epoch_acc = running_corrects.double() / data_size
        
        print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
    
    return model

# Finetuning the convnet

In [ ]:
model_pre = models.resnet18(pretrained=True)
model_pre.load_state_dict(torch.load('resnet18-5c106cde.pth')) # 加载预训练好的模型
model_conv = Net(model_pre)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model_conv.parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_pre = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=25)

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
epoch 0/24
Train Loss: 0.0595 Acc: 0.4848
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
epoch 1/24
Train Loss: 0.0294 Acc: 0.5133
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
epoch 2/24
Train Loss: 0.0233 Acc: 0.5336
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
epoch 3/24
Train Loss: 0.0175 Acc: 0.4978
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
epoch 4/24
Train Loss: 0.0170 Acc: 0.4355
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
epoch 5/24
Train Loss: 0.0133 Acc: 0.4079
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
epoch 6/24
Train Loss: 0.0068 Acc: 0.2998
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
epoch 7/24
Train Loss: 0.0002 Acc: 0.2698
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
epoch 8/24
Train Loss: 0.0001 Acc: 0.2698
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
epoch 9/24
T